In [2]:
import pandas as pd


In [25]:
df_groundtruth = pd.read_csv('../ClaimBuster_Datasets/datasets/groundtruth.csv')
df_all_sentences = pd.read_csv('../ClaimBuster_Datasets/datasets/all_sentences.csv')
df_crowdsourced = pd.read_csv('../ClaimBuster_Datasets/datasets/crowdsourced.csv')
df_crwd_grnd = pd.concat([df_groundtruth,df_crowdsourced],axis=0)
sentence_id_with_v = df_crwd_grnd['Sentence_id'].values
df_without_label = df_all_sentences[~((df_all_sentences['Speaker'] == 'Information') | (df_all_sentences['Speaker_title'].isna()) | (df_all_sentences['Sentence_id'].isin(sentence_id_with_v))) ]
df_without_label['Verdict'] = 7 #all non labels have 7 as verdict
df_full = pd.concat([df_crwd_grnd,df_without_label],axis=0)
df_full = df_full.sort_values('Sentence_id')
df_full.reset_index(drop=True,inplace=True)

# df_crwd_grnd.sort_values('Sentence_id').head()


C:\Users\mike1\AppData\Local\Temp\ipykernel_19180\3670880383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_label['Verdict'] = 7


In [26]:
def generate_word_dataset(df,presentence = 1,postsentence = 1):
    dataset = []
    label = []
    for index,row in df.iterrows():
        if index == 0 or index == len(df)-1:
            continue
        past = df.iloc[index-1]['Text']
        curr = row['Text']
        future = df.iloc[index+1]['Text']    
        if row['File_id'] != df.iloc[index-1]['File_id']:
            past = "EMPTY"
        if row['File_id'] != df.iloc[index+1]['File_id']:
            future = "EMPTY"
        dataset.append([past,curr,future])
        label.append(row['Verdict'])
    
    return dataset,label

data_text, labels = generate_word_dataset(df_full)

In [27]:
data_text[0],data_text[1] #can see each training sample has one past sentence curr sentence and one future sentence

(["I think we've seen a deterioration of values.",
  'I think for a while as a nation we condoned those things we should have condemned.',
  "For a while, as I recall, it even seems to me that there was talk of legalizing or decriminalizing marijuana and other drugs, and I think that's all wrong."],
 ['I think for a while as a nation we condoned those things we should have condemned.',
  "For a while, as I recall, it even seems to me that there was talk of legalizing or decriminalizing marijuana and other drugs, and I think that's all wrong.",
  "So we've seen a deterioration in values, and one of the things that I think we should do about it in terms of cause is to instill values into the young people in our schools."])